In [1]:
import gradio as gr

In [2]:
# Define a dictionary of chat word mappings
chat_words = {
    "AFAIK": "As Far As I Know",
    "AFK": "Away From Keyboard",
    "ASAP": "As Soon As Possible",
    "ATK": "At The Keyboard",
    "ATM": "At The Moment",
    "A3": "Anytime, Anywhere, Anyplace",
    "BAK": "Back At Keyboard",
    "BBL": "Be Back Later",
    "BBS": "Be Back Soon",
    "BFN": "Bye For Now",
    "B4N": "Bye For Now",
    "BRB": "Be Right Back",
    "BRT": "Be Right There",
    "BTW": "By The Way",
    "B4": "Before",
    "B4N": "Bye For Now",
    "CU": "See You",
    "CUL8R": "See You Later",
    "CYA": "See You",
    "FAQ": "Frequently Asked Questions",
    "FC": "Fingers Crossed",
    "FWIW": "For What It's Worth",
    "FYI": "For Your Information",
    "GAL": "Get A Life",
    "GG": "Good Game",
    "GN": "Good Night",
    "GMTA": "Great Minds Think Alike",
    "GR8": "Great!",
    "G9": "Genius",
    "IC": "I See",
    "ICQ": "I Seek you (also a chat program)",
    "ILU": "ILU: I Love You",
    "IMHO": "In My Honest/Humble Opinion",
    "IMO": "In My Opinion",
    "IOW": "In Other Words",
    "IRL": "In Real Life",
    "KISS": "Keep It Simple, Stupid",
    "LDR": "Long Distance Relationship",
    "LMAO": "Laugh My A.. Off",
    "LOL": "Laughing Out Loud",
    "LTNS": "Long Time No See",
    "L8R": "Later",
    "MTE": "My Thoughts Exactly",
    "M8": "Mate",
    "NRN": "No Reply Necessary",
    "OIC": "Oh I See",
    "PITA": "Pain In The A..",
    "PRT": "Party",
    "PRW": "Parents Are Watching",
    "QPSA?": "Que Pasa?",
    "ROFL": "Rolling On The Floor Laughing",
    "ROFLOL": "Rolling On The Floor Laughing Out Loud",
    "ROTFLMAO": "Rolling On The Floor Laughing My A.. Off",
    "SK8": "Skate",
    "STATS": "Your sex and age",
    "ASL": "Age, Sex, Location",
    "THX": "Thank You",
    "TTFN": "Ta-Ta For Now!",
    "TTYL": "Talk To You Later",
    "U": "You",
    "U2": "You Too",
    "U4E": "Yours For Ever",
    "WB": "Welcome Back",
    "WTF": "What The F...",
    "WTG": "Way To Go!",
    "WUF": "Where Are You From?",
    "W8": "Wait...",
    "7K": "Sick:-D Laugher",
    "TFW": "That feeling when",
    "MFW": "My face when",
    "MRW": "My reaction when",
    "IFYP": "I feel your pain",
    "TNTL": "Trying not to laugh",
    "JK": "Just kidding",
    "IDC": "I don't care",
    "ILY": "I love you",
    "IMU": "I miss you",
    "ADIH": "Another day in hell",
    "ZZZ": "Sleeping, bored, tired",
    "WYWH": "Wish you were here",
    "TIME": "Tears in my eyes",
    "BAE": "Before anyone else",
    "FIMH": "Forever in my heart",
    "BSAAW": "Big smile and a wink",
    "BWL": "Bursting with laughter",
    "BFF": "Best friends forever",
    "CSL": "Can't stop laughing"
}

In [3]:
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
import re
import string
stemmer = SnowballStemmer("english")
def extreme_clean(msg):
    new_msg = ''
    for word in msg.split():
        new_msg = new_msg + ' ' + stemmer.stem(word)
    msg = new_msg
    nopunc = [char for char in msg if char.lower() not in string.punctuation]
    nopunc = ''.join(nopunc)
    return [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]

def replace_chat_words(text):
    words = text.split()
    for i, word in enumerate(words):
        if word.lower() in chat_words:
            words[i] = chat_words[word.lower()]
    return ' '.join(words)
stop = stopwords.words('english')
def preprocess(msg: str):
    extreme_clean(msg)
    replace_chat_words(msg)
    msg = re.sub(r'[^a-zA-Z\s]', '', msg)
    msg = ' '.join([word for word in msg.split() if word not in (stop)])
    msg = msg.lower()
    msg = re.sub(r'\d+', '', msg)
    msg = re.sub(r'\s+', ' ', msg)
    msg = re.sub(r'[^\w\s]', '', msg)
    msg = re.sub(r'http\S+', '', msg)
    return msg 

In [4]:
import pickle
with open("svm.pkl", "rb") as f:
    svm = pickle.load(f)

In [5]:
with open("knn.pkl", "rb") as f:
    knn = pickle.load(f)

In [6]:
with open("rfc.pkl", "rb") as f:
    rfc = pickle.load(f)

In [7]:
with open("nb.pkl", "rb") as f:
    nb = pickle.load(f)

In [8]:
with open('tf-idf.pkl', 'rb') as f:
    tf_idf = pickle.load(f)

In [9]:
def get_sentiment(msg):
    return "Positive" if msg == 0 else "Negative"

In [18]:
import gradio as gr
def sentence_builder(msg, model):
    msg = preprocess(msg)
    msg = tf_idf.transform(bow.transform([msg]))
    if model == "SVM":
        return get_sentiment(svm.predict(msg))
    elif model == "KNN":
        return get_sentiment(knn.predict(msg))
    elif model == "RFC":
        return get_sentiment(rfc.predict(msg))
    elif model == "MultinomialNB":
        return get_sentiment(nb.predict(msg))

demo = gr.Interface(
    sentence_builder,
    [
        gr.Textbox(label="Enter Message: ", lines=3),
        gr.Dropdown(
            ["KNN", "SVM", "RFC", "MultinomialNB"], label="Select Model: "
        )
    ],
    [
        gr.Textbox(label="Prediction"),
    ]
)

if __name__ == "__main__":
    demo.launch()

Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.
